# Loading helper libraries 

In [36]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import re
import ast
import patsy
import datetime
import math

from pathlib import Path
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from plotnine import *
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse
from collections import defaultdict
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from plotnine.data import mpg
from mizani.formatters import percent_format



In [37]:
path = Path(os.getcwd())
tech_prep = os.path.join(str(path), "utils")
sys.path.append(tech_prep)
from py_helper_functions import *

# Loading Dataset 

### 1. main_df is the main dataset for my city, Lisbon.
### 2. amenities_cleaned_df is the dataset used for mapping amenities to my definitons

In [38]:
main_df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/DA3/main/Assignment%202/lisbon_listings.csv', delimiter=",",encoding="utf-8")
amenities_cleaned_df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/DA3/main/Assignment%202/unique_amenities_cleaned.csv', delimiter=",",encoding="utf-8")


In [39]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22605 entries, 0 to 22604
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            22605 non-null  int64  
 1   listing_url                                   22605 non-null  object 
 2   scrape_id                                     22605 non-null  int64  
 3   last_scraped                                  22605 non-null  object 
 4   source                                        22605 non-null  object 
 5   name                                          22605 non-null  object 
 6   description                                   22586 non-null  object 
 7   neighborhood_overview                         13429 non-null  object 
 8   picture_url                                   22605 non-null  object 
 9   host_id                                       22605 non-null 

#### Since our client  will be operating small and mid-size apartments hosting 2-6 guests. It makes sense to filter out listings that are a room or a shared unit and only keep the ones that a customer can book entirely

In [40]:
main_df = main_df[main_df["property_type"].str.contains(r'^Entire', regex=True)] # fitlering for entire property types
main_df = main_df.loc[(main_df['accommodates'] >= 2) & (main_df['accommodates'] <= 6)] # fitlering for 2-6 accommodates

In [41]:
main_df.head().T

,0,1,2,3,5
id,6499,25659,119120,29248,29396
listing_url,https://www.airbnb.com/rooms/6499,https://www.airbnb.com/rooms/25659,https://www.airbnb.com/rooms/119120,https://www.airbnb.com/rooms/29248,https://www.airbnb.com/rooms/29396
scrape_id,20230911050651,20230911050651,20230911050651,20230911050651,20230911050651
last_scraped,2023-09-11,2023-09-12,2023-09-12,2023-09-12,2023-09-12
source,city scrape,city scrape,city scrape,city scrape,city scrape
...,...,...,...,...,...
calculated_host_listings_count,1,1,8,5,1
calculated_host_listings_count_entire_homes,1,1,8,5,1
calculated_host_listings_count_private_rooms,0,0,0,0,0
calculated_host_listings_count_shared_rooms,0,0,0,0,0


# Data Cleaning and Data Preparation
1. cleaning `amenities` columns (grouping by category using amenities_cleaned_df) # this was done seperately as code was very long to include
2. getting number of bathrooms from `bathroom_texts` 

In [42]:
main_df['amenities'][0] #exploring our amenities columns

'["Shampoo", "River view", "Dishes and silverware", "Dining table", "Stove", "Coffee maker", "Paid street parking off premises", "Kitchen", "Microwave", "Freezer", "Bed linens", "Paid parking on premises", "Laundromat nearby", "Oven", "Host greets you", "Carbon monoxide alarm", "Mini fridge", "Ethernet connection", "TV with standard cable", "First aid kit", "Hot water", "Extra pillows and blankets", "Cooking basics", "Wifi", "Essentials", "Cleaning products", "Fire extinguisher", "Free street parking", "Room-darkening shades", "Free washer \\u2013 In unit", "Long term stays allowed"]'

In [43]:
# cleaning the amenities column
main_df["amenities"] = main_df["amenities"].astype(str).str.strip("[]").str.replace('"', "").str.split(",")

In [44]:
# cleaning the amenities column
old_value1 = '\\u2019n '
old_value2 = '\\u2013 '
old_value3 = '\\'
new_value = ''


main_df['amenities'] = main_df['amenities'].apply(lambda x: [item.replace(old_value1, new_value) for item in x])
main_df['amenities'] = main_df['amenities'].apply(lambda x: [item.replace(old_value2, new_value) for item in x])
main_df['amenities'] = main_df['amenities'].apply(lambda x: [item.replace(old_value3, new_value) for item in x])



In [45]:
# replacing and mapping values as defined by me in the unique cleaned amenities dataset

replacement_dict = dict(zip(amenities_cleaned_df['original'], amenities_cleaned_df['new']))

main_df['amenities_cleaned'] = main_df['amenities'].apply(lambda x: [replacement_dict[val] if val in replacement_dict else val for val in x])

print(sorted(main_df["amenities_cleaned"].explode().unique()))

['', 'airconditioning', 'backyard', 'bbq', 'beachaccess', 'bodysoap', 'breakfast', 'cabletv', 'closet', 'coffeemaker', 'conditioner', 'dishwasher', 'dryer', 'elevator', 'essentials', 'fireextinguisher', 'firstaidkit', 'freeparking', 'gameconsole', 'gym', 'hairdryer', 'hangers', 'hottub', 'indoorfireplace', 'keypad', 'kidfriendly', 'kitchen', 'lockbox', 'luxuryamenities', 'others', 'outdooramenities', 'paidparking', 'petfriendly', 'pool', 'privateentrance', 'privatelivingroom', 'refrigerator', 'securitycameras', 'selfcheckin', 'shampoo', 'smokedetector', 'smokingallowed', 'soundsystem', 'tv', 'washer', 'wifi', 'windowview', 'workspace']


In [46]:
# extracting bathrooms from bathroom text

def extract_bathrooms(text):
    match = re.match(r'^([\d.]+)\s*baths?', str(text))
    if match:
        return float(match.group(1))
    else:
        return 0

main_df['n_bathrooms'] = main_df['bathrooms_text'].apply(extract_bathrooms)

#### Filtering out irrelevant columns and renaming the relevant ones for better information representing

In [47]:
columns_to_keep = ['property_type','room_type','price','accommodates', 'beds','bedrooms','n_bathrooms',
                   'review_scores_rating', 'host_is_superhost','number_of_reviews', 'minimum_nights', 'maximum_nights', 
                   'reviews_per_month','neighbourhood_cleansed','neighbourhood_group_cleansed','instant_bookable','amenities_cleaned']

filtered_df= main_df[columns_to_keep]

column_renaming = {
    'property_type':'f_property_type',
    'room_type':'f_room_type',
    'accommodates': 'n_accommodates',
    'beds': 'n_beds',
    'bedrooms':'n_bedrooms',
    'minimum_nights': 'n_minimum_nights',
    'maximum_nights': 'n_maximum_nights',
    'instant_bookable': 'd_instant_bookable',
    'host_is_superhost':'d_host_is_superhost',
    'number_of_reviews': 'n_number_of_reviews',
    'reviews_per_month': 'n_reviews_per_month',
    'review_scores_rating': 'n_review_scores_rating',
    'neighbourhood_cleansed': 'f_neighbourhood_cleansed',
    'neighbourhood_group_cleansed': 'f_neighbourhood_group_cleansed',    
}

filtered_df.rename(columns=column_renaming, inplace=True)

#### Transforming variables and cleaning our dependant variable
1. `property type` to category
2. `room type` to category
3. `neighbourhood cleansed` to category
4. `instant bookable` to 1 0
5. `host is super` host to 1 0
6. `price` cleaning

In [48]:

filtered_df["f_property_type"] = filtered_df["f_property_type"].astype("category") # category column for modelling
filtered_df["f_room_type"] = filtered_df["f_room_type"].astype("category") # category column for modelling
filtered_df["f_neighbourhood_cleansed"] = filtered_df["f_neighbourhood_cleansed"].astype("category") # category column for modelling
filtered_df["f_neighbourhood_group_cleansed"] = filtered_df["f_neighbourhood_group_cleansed"].astype("category") # category column for modelling


filtered_df['d_instant_bookable'] = np.where(filtered_df['d_instant_bookable'] == 't', '1', '0') .astype(int) # changing to 1 and 0
filtered_df['d_host_is_superhost'] = np.where(filtered_df['d_host_is_superhost'] == 't', '1', '0') .astype(int) # changing to 1 and 0


# cleaning price variable
filtered_df['price'] = filtered_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
filtered_df = filtered_df.dropna(subset=['price'])
filtered_df = filtered_df[filtered_df['price'] > 0]
filtered_df['price'] = filtered_df['price'].astype(int) 